In [1]:
import pandas as pd
import openpyxl

In [2]:
tizada = pd.read_excel('Consulta Listado de Tizada.xls')

WARNING *** file size (10772352) not 512 + multiple of sector size (512)


In [3]:
tizada = tizada.iloc[1:-1, :] #Elimino primera y ultima fila
tizada['Unnamed: 7'] = tizada['Unnamed: 7'].replace(['Descripcion'], ['Tela detalle'])
tizada['Unnamed: 10'] = tizada['Unnamed: 10'].replace(['Corte'], ['Fecha'])
tizada = tizada.rename(columns=tizada.iloc[0]).drop(tizada.index[0]) #Primera fila pasa a ser header 
tizada = tizada.drop(columns=['Familia']) #Elimino columnas que no necesito
tizada['IdTela'] = tizada['IdTela'].fillna('Acomp') # Reemplazo nan por 'Acomp'
temporadas = ['V22', 'I22', 'V23', 'I23'] 
tizadaPorTemporada = tizada.reindex(columns=['Orden', 'aux', 'aux 2', 'Producto', 'Descripcion', 'Temporada', 'Fecha', 'Color',
                                 'IdTela', 'Tela' , 'Tela detalle', 'Tizada', 'Consumo', 'Cortado'])
tizadaPorTemporada = tizadaPorTemporada.apply(lambda x: x.astype(str).str.strip() if x.dtype == "object" else x) #Quito espacios en blanco de las celdas
tizadaPorTemporada = tizadaPorTemporada.loc[tizadaPorTemporada['Temporada'].isin(temporadas)]
tizadaPorTemporada['aux'] = tizadaPorTemporada.groupby('Orden').cumcount() + 1
tizadaPorTemporada['aux 2'] = tizadaPorTemporada['Orden'].astype(str) + tizadaPorTemporada['aux'].astype(str)
tizadaPorTemporada[["Orden", "aux", "aux 2"]] = tizadaPorTemporada[["Orden", "aux", "aux 2"]].apply(pd.to_numeric, downcast="integer")
tizadaPorTemporada[["Tizada", "Consumo", "Cortado"]] = tizadaPorTemporada[["Tizada", "Consumo", "Cortado"]].apply(pd.to_numeric, downcast="float")

In [4]:
with pd.ExcelWriter(r'\\NasConbra011\Administra\Reportes\Tizada.xlsx') as writer:
    tizadaPorTemporada.to_excel(writer, sheet_name='Tizada', index=False)